In [19]:
import pandas as pd
df = pd.read_csv("customer_sku_monthly_summary.csv")
df["OrderMonth"] = pd.to_datetime(df["OrderMonth"])
df.sort_values(by=["Customer", "OrderMonth"], inplace= True)
df.head()

,Customer,OrderMonth,SKU_Count
0,399 Vietnamese Kitchen,2025-01-01,6
1,399 Vietnamese Kitchen,2025-02-01,3
2,399 Vietnamese Kitchen,2025-03-01,11
3,399 Vietnamese Kitchen,2025-05-01,8
4,626 Commissary,2025-07-01,4


In [20]:
df["LastMonth_SKU"] = df.groupby("Customer")["SKU_Count"].shift(1)
df["SKUPct"] = ((df["SKU_Count"] - df["LastMonth_SKU"])/df["LastMonth_SKU"]).round(3)
df.rename(columns={"SKUPct": "ChangePct"}, inplace=True)

df.head(10
       )

,Customer,OrderMonth,SKU_Count,LastMonth_SKU,ChangePct
0,399 Vietnamese Kitchen,2025-01-01,6,NaN,NaN
1,399 Vietnamese Kitchen,2025-02-01,3,6.0,-0.500
2,399 Vietnamese Kitchen,2025-03-01,11,3.0,2.667
3,399 Vietnamese Kitchen,2025-05-01,8,11.0,-0.273
4,626 Commissary,2025-07-01,4,NaN,NaN
5,626 Hospitality Group,2025-05-01,3,NaN,NaN
6,American KGP Inc.,2025-03-01,2,NaN,NaN
7,And Destroy Coffee,2025-07-01,8,NaN,NaN
8,Awakening,2025-05-01,7,NaN,NaN
9,Awakening,2025-06-01,6,7.0,-0.143


In [21]:
# 初始化新列
df['DropStreak'] = 0
df['RiseStreak'] = 0

# 对每个客户进行遍历
for customer in df['Customer'].unique():
    streak_drop = 0
    streak_rise = 0
    # 筛出该客户的索引
    customer_idx = df[df['Customer'] == customer].index
    for idx in customer_idx:
        current = df.at[idx, 'SKU_Count']
        last = df.at[idx, 'LastMonth_SKU']
        
        if pd.isna(last):
            # 没有上月数据，归零
            streak_drop = 0
            streak_rise = 0
        else:
            if current < last:
                streak_drop += 1
                streak_rise = 0
            elif current > last:
                streak_rise += 1
                streak_drop = 0
            else:
                streak_drop = 0
                streak_rise = 0
        
        # 赋值给当前行
        df.at[idx, 'DropStreak'] = streak_drop
        df.at[idx, 'RiseStreak'] = streak_rise



In [22]:
# 添加行为标签
df["SKUPct"] = ((df["SKU_Count"] - df["LastMonth_SKU"]) / df["LastMonth_SKU"]).round(3)

def tag_alert(row):
    if pd.isna(row['ChangePct']):
        return ''
    if row['ChangePct'] < -0.5 or row['DropStreak'] >= 2:
        return '⚠️'
    elif row['ChangePct'] > 0.5 or row['RiseStreak'] >= 2:
        return '✨'
    else:
        return ''

df['Alert'] = df.apply(tag_alert, axis=1)

# 添加颜色字段（给 Tableau 使用）
def color_tag(row):
    if row['Alert'] == '⚠️':
        return 'Red'
    elif row['Alert'] == '✨':
        return 'Green'
    else:
        return 'Gray'

df['ColorTag'] = df.apply(color_tag, axis=1)

# 查看结果
df[['Customer', 'OrderMonth', 'SKU_Count', 'ChangePct', 'DropStreak', 'RiseStreak', 'Alert', 'ColorTag']].head(10)



,Customer,OrderMonth,SKU_Count,ChangePct,DropStreak,RiseStreak,Alert,ColorTag
0,399 Vietnamese Kitchen,2025-01-01,6,NaN,0,0,,Gray
1,399 Vietnamese Kitchen,2025-02-01,3,-0.500,1,0,,Gray
2,399 Vietnamese Kitchen,2025-03-01,11,2.667,0,1,✨,Green
3,399 Vietnamese Kitchen,2025-05-01,8,-0.273,1,0,,Gray
4,626 Commissary,2025-07-01,4,NaN,0,0,,Gray
5,626 Hospitality Group,2025-05-01,3,NaN,0,0,,Gray
6,American KGP Inc.,2025-03-01,2,NaN,0,0,,Gray
7,And Destroy Coffee,2025-07-01,8,NaN,0,0,,Gray
8,Awakening,2025-05-01,7,NaN,0,0,,Gray
9,Awakening,2025-06-01,6,-0.143,1,0,,Gray


In [25]:
output_path = r"./sku_variation_analysis_clean.csv"
df.to_csv(output_path, index=False)

